# Lesson 2: Sequential Chats and Customer Onboarding

### A call center use case for a financial services firm that sells services like credit cards, checking accounts, etc. In this example, the customer has questions about travel credit cards to which the agents respond intelligently.

## Setup

In [9]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
from dotenv import load_dotenv

load_dotenv()


config_list = [
  
    {
        'model': 'gpt4',
        'api_key': ''+(os.environ.get('AZURE_OPENAI_KEY'))+'',
        'base_url':  ''+(os.environ.get('AZURE_OPENAI_ENDPOINT'))+'',
        'api_type': 'azure',
        'api_version': '2024-02-01'
    }
]

In [33]:
from autogen import ConversableAgent

## Creating the needed agents

In [34]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a Personal Info Agent. Your task is to collect and manage personal information from customers. 
    Ensure that you handle data sensitively and securely, complying with privacy regulations such as GDPR. 
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config={
        "config_list": config_list,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [35]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are an Onboarding Topic Preference Agent. Your role is to assist new customers in selecting their 
    preferred topics and interests for a personalized onboarding experience. Ask the customers about their 
    interests, preferences, and any specific topics they would like to focus on. Provide recommendations based 
    on their responses and ensure that the onboarding experience is tailored to their needs.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
      llm_config={
        "config_list": config_list,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [36]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a Customer Engagement Agent in Financial Services company that provides services like credit cards, checking accounts, etc. Your job is to maintain and enhance customer engagement through 
    personalized interactions. You should ask customers about their recent experiences, gather feedback, 
    and suggest relevant products or services based on their preferences and previous interactions. 
    Your goal is to keep customers engaged and satisfied with their experience.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
      llm_config={
        "config_list": config_list,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [37]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [38]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what products you are "
                "interested in learning about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 3,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find more details of the product you are interested in.",
        "max_turns": 3,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different inputs, such as name, location, and preferences.

In [39]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


c:\Users\akaruparti\anaconda\Lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to Onboarding Personal Information Agent):

anurag from new york

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Great! Thank you for providing your name and location, Anurag from New York. We have collected all necessary information. TERMINATE

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

thanks

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what products you are interested in learning about?
Context: 
{
  "name": "Anurag",
  "location": "New York"
}

---------

## Print out the summary

In [40]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Anurag",
  "location": "New York"
}


Anurag, who is from New York, wants to learn about travel credit cards, specifically about maximizing travel rewards, including information on air miles, hotel loyalty points and dining rewards.


Anurag, from New York, is interested in applying for the Globetrotter Platinum travel credit card which provides 3 air miles for every dollar spent on airlines, 2 hotel loyalty points for every dollar spent on accommodations and 5x dining rewards. The card also has other benefits like priority boarding, complimentary airport lounge access, no foreign transaction fees along with a welcome bonus of 30,000 air miles after spending $3,000 in the first 3 months.




## Print out the cost

In [41]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.018659999999999996, 'gpt-4-32k': {'cost': 0.018659999999999996, 'prompt_tokens': 219, 'completion_tokens': 46, 'total_tokens': 265}}, 'usage_excluding_cached_inference': {'total_cost': 0}}


{'usage_including_cached_inference': {'total_cost': 0.04548, 'gpt-4-32k': {'cost': 0.04548, 'prompt_tokens': 528, 'completion_tokens': 115, 'total_tokens': 643}}, 'usage_excluding_cached_inference': {'total_cost': 0.04548, 'gpt-4-32k': {'cost': 0.04548, 'prompt_tokens': 528, 'completion_tokens': 115, 'total_tokens': 643}}}


{'usage_including_cached_inference': {'total_cost': 0.20975999999999997, 'gpt-4-32k': {'cost': 0.20975999999999997, 'prompt_tokens': 2076, 'completion_tokens': 710, 'total_tokens': 2786}}, 'usage_excluding_cached_inference': {'total_cost': 0.20975999999999997, 'gpt-4-32k': {'cost': 0.20975999999999997, 'prompt_tokens': 2076, 'completion_tokens': 710, 'total_tokens': 2786}}}


